# Cuaderno 2: Reducción de Dimensión, BNMF y Clustering

**Continuación directa del Cuaderno 1: utiliza los datos limpios y procesados exportados previamente.**

---

## Diagrama de Flujo del Cuaderno

```
1. Estandarización (StandardScaler)
   ↓
2. PCA (Análisis de Componentes Principales)
   ↓
3. BNMF (Bayesian Non-Negative Matrix Factorization)
   ↓
4. Clustering (k-Means / GMM)
   ↓
5. Visualización y Exportación
```

## 1. Estandarización de los Datos
Se cargan los datos limpios del cuaderno 1 y se aplica StandardScaler.

In [ ]:
# Cargar datos limpios del cuaderno 1
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('processed_data/data_prepared.csv')
X = df.drop(columns=['hospital_expire_flag', 'data_split'], errors='ignore')

# Estandarización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 2. PCA: Análisis de Componentes Principales
Se calcula la varianza explicada y se selecciona el número óptimo de componentes.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Varianza explicada
explained_var = pca.explained_variance_ratio_
plt.figure(figsize=(8,4))
plt.plot(range(1, len(explained_var)+1), explained_var.cumsum(), marker='o')
plt.xlabel('N° de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('PCA Explained Variance')
plt.grid(True)
plt.savefig('pca_explained_variance.eps', format='eps')
plt.show()

## 3. BNMF: Bayesian Non-Negative Matrix Factorization
Se aplica BNMF, se obtienen matrices W y H, y se interpretan los componentes.

In [ ]:
# Instala scikit-learn-extra si no lo tienes: pip install scikit-learn-extra
from sklearn.decomposition import NMF

n_components = 5  # Ajusta según tu análisis
nmf = NMF(n_components=n_components, init='nndsvda', random_state=42, solver='mu', beta_loss='kullback-leibler', max_iter=500)
W = nmf.fit_transform(abs(X_scaled))
H = nmf.components_

print('Forma de W:', W.shape)
print('Forma de H:', H.shape)

### Interpretación de Componentes (W y H)
Explica qué representa cada componente y muestra los pesos principales.

In [ ]:
import numpy as np
for i, comp in enumerate(H):
    print(f'Componente {i+1}:')
    top_features = np.argsort(comp)[-5:][::-1]
    print('  Principales variables:', X.columns[top_features].tolist())

## 4. Clustering sobre Componentes
Se aplica k-Means y/o GMM sobre los componentes de BNMF y PCA, y se compara el desempeño.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# Clustering sobre BNMF
kmeans_bnmf = KMeans(n_clusters=3, random_state=42).fit(W)
labels_bnmf = kmeans_bnmf.labels_

# Clustering sobre PCA
kmeans_pca = KMeans(n_clusters=3, random_state=42).fit(X_pca[:, :n_components])
labels_pca = kmeans_pca.labels_

## 5. Visualización de Clusters y Componentes
Gráficas en el espacio reducido y distribución de componentes.

In [ ]:
import seaborn as sns

# Visualización en espacio BNMF
plt.figure(figsize=(8,6))
sns.scatterplot(x=W[:,0], y=W[:,1], hue=labels_bnmf, palette='Set1')
plt.title('Clusters en espacio BNMF')
plt.savefig('clusters_bnmf.eps', format='eps')
plt.show()

# Visualización en espacio PCA
plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=labels_pca, palette='Set2')
plt.title('Clusters en espacio PCA')
plt.savefig('clusters_pca.eps', format='eps')
plt.show()

## 6. Comparación y Conclusiones
Compara los resultados de clustering y discute cuál técnica representa mejor la estructura de los datos.